In [32]:
from IPython.display import clear_output

import datetime
import time
import os


from tools import (
    play_beep,
    print_account_summary,
    print_order,
    print_all_openorders,
    print_openorders,
)

from IPython.display import display
from IPython.core.display import HTML

play_beep()

display(HTML("<style>.output_subarea { overflow: auto; }</style>"))

In [34]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=13)
util.logToFile(f"{datetime.datetime.now().strftime('%Y-%m-%d')}.log")

2024-05-03 07:33:20,422 ib_async.client INFO Connecting to 127.0.0.1:4001 with clientId 13...
2024-05-03 07:33:20,425 ib_async.client INFO Connected
2024-05-03 07:33:20,437 ib_async.client INFO Logged on to server version 176
2024-05-03 07:33:20,441 ib_async.client INFO API connection ready
2024-05-03 07:33:20,458 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfuture
2024-05-03 07:33:20,463 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-05-03 07:33:20,477 ib_async.wrapper INFO Warning 2107, reqId -1: HMDS data farm connection is inactive but should be available upon demand.ushmds
2024-05-03 07:33:20,483 ib_async.wrapper INFO Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
2024-05-03 07:33:20,544 ib_async.wrapper INFO position: Position(account='U2340948', contract=Stock(conId=4762, symbol='BA', exchange='NYSE', currency='USD', localSymbol='BA', tradingClass='BA'), position=254.0, avgCost

2024-05-03 07:33:35,442 ib_async.wrapper ERROR Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. The following farms are connected: usfuture; usfarm; secdefil. The following farms are not connected: ushmds.
2024-05-03 07:33:55,978 ib_async.wrapper INFO orderStatus: Trade(contract=Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=LimitOrder(orderId=170, clientId=13, permId=71605396, action='BUY', totalQuantity=1.0, lmtPrice=17903.0, auxPrice=0.0, account='U10394496'), orderStatus=OrderStatus(orderId=170, status='Submitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=71605396, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 3, 12, 33, 55, 636210, tzinfo=datetime.timezone.utc), status='PendingSubmit', 

In [35]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

# ib.cancelMktDepth(contract)

In [36]:
util.tree(ib.positions()[11])

{'account': 'U10394496',
 'contract': {'Future': {'secType': 'FUT',
   'conId': 620730920,
   'symbol': 'NQ',
   'lastTradeDateOrContractMonth': '20240621',
   'multiplier': '20',
   'currency': 'USD',
   'localSymbol': 'NQM4',
   'tradingClass': 'NQ'}},
 'position': -5.0,
 'avgCost': 355059.8214844}

In [37]:
# default reset values
open_trade = None
close_trade = None
# get a specific order by id

# trades = get_all_openorders(ib)
# open_trade = [trade for trade in trades if trade.order.permId == 449873795]
# print_order(open_trade)

# future = [
#     pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
# ][0]

# print(future)
print()
print_account_summary(ib=ib)
print()
print_openorders(ib=ib)


AccountType : INDIVIDUAL
Cushion : 0.37232
LookAheadNextChange : 1714743000
AccruedCash : 0.00
AvailableFunds : 63625.23
BuyingPower : 303200.41
EquityWithLoanValue : 80431.48
ExcessLiquidity : 76484.46
FullAvailableFunds : 63625.23
FullExcessLiquidity : 76484.46
FullInitMarginReq : 138504.93
FullMaintMarginReq : 125695.70
GrossPositionValue : 3869.93
InitMarginReq : 138504.93
LookAheadAvailableFunds : 103802.30
LookAheadExcessLiquidity : 113009.06
LookAheadInitMarginReq : 98327.86
LookAheadMaintMarginReq : 89171.10
MaintMarginReq : 125695.70
NetLiquidation : 205426.84
PreviousDayEquityWithLoanValue : 4266.00
RegTEquity : 4248.01
RegTMargin : 3947.02
SMA : 623936.55
TotalCashValue : 202180.16



In [38]:
# t1 = get_all_openorders(ib)[1]
# print(t1)

open_trade = None
close_trade = None

In [44]:
## SELL TO OPEN SCALP 0.1
########################################

sell_10_10_scalp = {
    "strategy": "SELL TO OPEN SCALP 10/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10,
    "pause_seconds": 30,
}

buy_scalp = {
    "strategy": "BUY TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -15,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10,
    "pause_seconds": 30,
}

buy_7_11_scalp = {
    "strategy": "BUY TO OPEN SCALP 10/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -7,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 11,
    "pause_seconds": 30,
}

strategy_details = buy_scalp

print(strategy_details["strategy"])

while True:
    clear_output(wait=True)

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details["open_action"]
        qty = strategy_details["open_qty"]

        if strategy_details["open_ref"] == "bid":
            price_ref = ticker.domBids[0].price
        elif strategy_details["open_ref"] == "ask":
            price_ref = ticker.domAsks[0].price
        elif strategy_details["open_ref"] == "mid":
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
        elif strategy_details["open_ref"] == "last":
            raise Exception("Not implemented")

        lmtPrice = (
            price_ref
            + strategy_details["open_ticks"] * strategy_details["tick_increment"]
        )
        print(
            f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
        )
        print()

        if strategy_details["open_type"] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception("Not implemented")

        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()

    print("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            print(
                f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})\n"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(
                seconds=strategy_details["pause_seconds"]
            ):
                print("Cancelling order due to timeout:")
                ib.cancelOrder(open_trade.order)
                ib.sleep(1)
                print()

        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details["close_action"]
            qty = strategy_details["close_qty"]

            if strategy_details["close_ref"] == "open_price_fill":
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details["close_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["close_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["close_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            elif strategy_details["close_ref"] == "last":
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["close_ticks"] * strategy_details["tick_increment"]
            )
            print(
                f"Placing close trade: {action}, {strategy_details['close_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
            )

            if strategy_details["close_type"] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            close_trade = ib.placeOrder(contract, close_order)
            ib.sleep(1)
            play_beep(500, 500)

        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            print("***** order is inactive *****")
            print(open_trade.log)
            print("*****************************")
            open_trade = None

    print(f"CLOSE ORDER::")
    print_order(close_trade)
    print()

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep(2500, 500)
            print(
                "Close trade filled @ {}\n".format(close_trade.orderStatus.avgFillPrice)
            )
            open_trade = None
            close_trade = None
            ib.accountSummary()

            # small break before next trade
            for i in range(5):
                print(".", end="")
                ib.sleep(1)

    print("ALL OPEN ORDERS::")
    print_openorders(ib=ib)
    print()

    if ticker is not None and ticker.domBids is not None and ticker.domAsks is not None:
        for i in range(min(len(ticker.domBids), len(ticker.domAsks))):
            bid_size = ticker.domBids[i].size
            bid_price = ticker.domBids[i].price
            ask_price = ticker.domAsks[i].price
            ask_size = ticker.domAsks[i].size
            print(f"{bid_size:>8} {bid_price:>10} | {ask_price:<10} {ask_size:<8}")

    print()
    fills = [t.fills for t in ib.trades() if t.fills != []]
    executions = [f[0].execution for f in fills]
    print(f"Day Executions: {len(executions)}")

    future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"]
    for f in future:
        print(
            f"{f.contract.symbol} {f.position} @ {f.avgCost/float(contract.multiplier)}"
        )
    print()
    print_account_summary(ib=ib)
    ib.sleep(3)

OPEN ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	71605413	Filled	BUY	1.0	0.0		17952.5	

CLOSE ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	71605414	Submitted	SELL	0.0	1.0		17955.25	

ALL OPEN ORDERS::
71605414	SELL	1.0	17955.25	

     2.0   17928.25 | 17929.25   2.0     
     3.0    17928.0 | 17929.5    3.0     
     5.0   17927.75 | 17929.75   5.0     
     4.0    17927.5 | 17930.0    3.0     
     7.0   17927.25 | 17930.25   2.0     

Day Executions: 11
NQ -4.0 @ 17850.0518

AccountType : INDIVIDUAL
Cushion : 0.479383
LookAheadNextChange : 1714743000
AccruedCash : 0.00
AvailableFunds : 86007.34
BuyingPower : 383072.81
EquityWithLoanValue : 100366.76
ExcessLiquidity : 96410.01
FullAvailableFunds : 86007.34
FullExcessLiquidity : 96410.01
FullInitMarginReq : 111968.02
FullMaintMarginReq : 101565.35
GrossPositionValue : 3773.97
InitMarginReq : 111968.02
LookAheadAvailableFunds : 118218.18
LookAheadExcessLiquidity : 125692.59
LookAheadInitMargi

KeyboardInterrupt: 

In [ ]:
fills = [t.fills for t in ib.trades() if t.fills != []]
executions = [f[0].execution for f in fills]
util.df(executions)

In [ ]:
orders = ib.openOrders()
orders.

In [ ]:
print_all_openorders(ib=ib)

In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )

In [ ]:
trades_df = util.df(ib.trades())
trades_df

In [ ]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("trades.db")

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create the "trades" table if it doesn't exist
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS trades (
        contract TEXT,
        order TEXT,
        orderStatus TEXT,
        fills TEXT,
        log TEXT,
        advancedError TEXT
    )
"""
)

# Convert the DataFrame to a list of tuples
trades_data = trades_df.to_records(index=False).tolist()

# Insert the data into the "trades" table
cursor.executemany(
    """
    INSERT INTO trades (contract, order, orderStatus, fills, log, advancedError)
    VALUES (?, ?, ?, ?, ?, ?)
""",
    trades_data,
)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')